<a href="https://colab.research.google.com/github/gverafei/scraping/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Investigating how to scrape the web**
March 2025

## Configure virtual environment

Solo se ejecuta lo siguiente la primera vez. Pedirá seleccionar the kernel from the upper right corner. Choose this virtual environment we just created.

In [ ]:
!python3 -m venv .venv
!source .venv/bin/activate # Linux/Mac
# !.\venv\Scripts\activate # Windows

Inicializa el repositorio en GitHub. Todo esto se hace desde la terminal.

In [ ]:
# !git init
# !git remote add origin https://github.com/gverafei/scraping.git
# !git pull origin main
# git add .
# git commit -m "Initial commit"
# git push --set-upstream origin main

In [ ]:
!pip install --upgrade pip --quiet

Install required packages

```bash
pip install html2text
pip install markdownify
```

## Create the initial data

In [1]:
competitor_sites = [
    {
        "name": "Amazon",
        "url": "https://www.amazon.com"
    },
    {
        "name": "UV",
        "url": "https://www.uv.mx"
    },
    {
        "name": "W3C ACT Rules",
        "url": "https://www.w3.org/WAI/standards-guidelines/act/rules/"
    },
]

## Setup cost's calculations

The idea is to compare side-by-side.

We can calculate how much it'll cost by using OpenAI's `tiktoken` library from: https://github.com/openai/tiktoken

And costs from: https://openai.com/api/pricing/

In [2]:
!pip install tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00


In [3]:
import tiktoken

def count_tokens(input_string: str) -> int:
    encoder = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoder.encode(input_string)
    return len(tokens)

def calculate_cost(input_string: str, cost_per_million_tokens: float = 2.5) -> float:
    num_tokens = count_tokens(input_string)
    total_cost = (num_tokens / 1_000_000) * cost_per_million_tokens
    return total_cost

# Example usage:
input_string = "Porque la gallina cruzó el camino? Pues porque quería llegar al otro lado."
cost = calculate_cost(input_string)
print(f"The total cost for using gpt-4o is: $US {cost:.6f}")

The total cost for using gpt-4o is: $US 0.000043


## Table para ver los resultados

Ahora, para ver los resultados de la comparaciones, instalamos un paquete para ver tablas en línea de comandos: https://pypi.org/project/prettytable/

Y también instalamos un paquete para ver una barra de progreso bonita en loops: https://pypi.org/project/tqdm/

In [4]:
!pip install prettytable --quiet

In [ ]:
!pip install tqdm --quiet

In [5]:
from typing import List, Callable, Dict
from prettytable import PrettyTable
from tqdm import tqdm

def view_scraped_content(scrape_url_functions: List[Dict[str, Callable[[str], str]]], sites_list: List[Dict[str, str]], characters_to_display: int = 500, table_max_width: int = 50) -> List[Dict[str, str]]:
    content_table_headers = ["Site Name"] + [f"{func['name']} content" for func in scrape_url_functions]
    cost_table_headers = ["Site Name"] + [f"{func['name']} cost" for func in scrape_url_functions]

    content_table = PrettyTable()
    content_table.field_names = content_table_headers

    cost_table = PrettyTable()
    cost_table.field_names = cost_table_headers

    scraped_data = []

    for site in sites_list:
        content_row = [site['name']]
        cost_row = [site['name']]
        site_data = {"provider": site['name'], "sites": []}

        for scrape_function in scrape_url_functions:
            function_name = scrape_function['name']
            for _ in tqdm([site], desc=f"Processing site {site['name']} using {function_name}"):
                try:
                    content = scrape_function['function'](site['url'])
                    content_snippet = content[:characters_to_display]
                    content_snippet = f"{len(content):,} characters retrieved:\n\n" + content_snippet
                    content_row.append(content_snippet)

                    cost = calculate_cost(content)
                    cost_row.append(f"${cost:.6f}")

                    site_data["sites"].append({"name": function_name, "content": content})
                except Exception as e:
                    error_message = f"Error: {str(e)}"
                    content_row.append(error_message)
                    cost_row.append("Error")

                    site_data["sites"].append({"name": function_name, "content": error_message})
                    continue

        content_table.add_row(content_row)
        cost_table.add_row(cost_row)
        scraped_data.append(site_data)

    content_table.max_width = table_max_width
    content_table.hrules = True

    cost_table.max_width = table_max_width
    cost_table.hrules = True

    print("Content Table:")
    print(content_table)

    print("\nCost Table:\nThis is how much it would cost to use gpt-4o to parse this content for extraction.")
    print(cost_table)

    return scraped_data

## Setup all the scrapers

Let's setup all of our scrapers.

## Beautiful Soup

Se instala este paquete desde: https://pypi.org/project/beautifulsoup4/

Y también request para hacer peticiones desde: https://pypi.org/project/requests/

In [7]:
!pip install requests beautifulsoup4 --quiet

In [8]:
# Beautiful Soup utility functions

import requests
from bs4 import BeautifulSoup

def beautiful_soup_scrape_url(url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return str(soup)

## Reader API by Jina AI

Este es especialmente para LLMs también. Setup Jina AI's scrape method from: https://jina.ai/reader/

In [9]:
import requests

def scrape_jina_ai(url: str) -> str:
  headers = {
        'X-Return-Format': 'html',
        'X-Wait-For-Selector': '.slick-initialized',
        'X-With-Images-Summary': 'all'
  }
  data = {
      'url': url,
      'injectPageScript': [
          'window.scrollTo(0, document.body.scrollHeight)'
      ]
  }

  response = requests.post('https://r.jina.ai/', headers=headers, json=data)
  return response.text

## Playwright

La manera clásica de hacer scraping. No es especial para LLMs desde: https://playwright.dev/

In [10]:
!pip install playwright --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 18.4 MB/s eta 0:00:00


In [11]:
!playwright install

164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 28.5s164.9 MiB [] 0% 28.3s164.9 MiB [] 0% 17.3s164.9 MiB [] 0% 11.3s164.9 MiB [] 0% 8.3s164.9 MiB [] 1% 7.2s164.9 MiB [] 1% 6.7s164.9 MiB [] 2% 6.3s164.9 MiB [] 2% 6.7s164.9 MiB [] 2% 6.6s164.9 MiB [] 3% 6.3s164.9 MiB [] 3% 6.4s164.9 MiB [] 3% 5.8s164.9 MiB [] 4% 5.3s164.9 MiB [] 4% 5.5s164.9 MiB [] 4% 5.3s164.9 MiB [] 5% 5.1s164.9 MiB [] 6% 4.8s164.9 MiB [] 6% 4.7s164.9 MiB [] 6% 5.0s164.9 MiB [] 6% 5.4s164.9 MiB [] 6% 5.6s164.9 MiB [] 7% 5.4s164.9 MiB [] 7% 5.2s164.9 MiB [] 8% 4.9s164.9 MiB [] 9% 4.8s164.9 MiB [] 10% 4.7s164.9 MiB [] 10% 4.6s164.9 MiB [] 11% 4.5s164.9 MiB [] 11% 4.4s164.9 MiB [] 12% 4.4s164.9 MiB [] 13% 4.3s164.9 MiB [] 13% 4.4s164.9 MiB [] 14% 4.3s164.9 MiB [] 14% 4.2s164.9 MiB [] 15% 4.1s164.9 MiB [] 16% 3.9s164.9 MiB [] 17% 3.8s164.9 MiB [] 17% 3.7s164.9 MiB [] 18% 3.7s164.9 MiB [] 18% 3.6s164.9 MiB [] 19% 3.5s164.9 MiB [] 20% 3.4s164.9 MiB [] 21% 3.3s164.9 MiB [] 22% 3.3s164.9 MiB [] 23% 3.2s164.9 MiB [] 23% 3.1s164.9 MiB [] 24

In [12]:
!pip install nest_asyncio --quiet

In [13]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright

async def scrape_playwright(url: str):
    async with async_playwright() as pw:
        browser =  await pw.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        # Espera para que cargue la pagina
        await page.wait_for_load_state('domcontentloaded')
        # Ejecuta un script para bajar hasta el final de la pagina
        # await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
        # Tambien se puede hacer con el teclado
        await page.keyboard.press('End')
        # Espera a que baje el scroll
        await page.wait_for_timeout(2000)

        html = await page.content()
        await browser.close()

        return html

def scrape_playwright_sync(url: str):
    return asyncio.run(scrape_playwright(url))

## Crawl4AI: Open-Source LLM-Friendly Web Crawler & Scraper

Este es especialmente para obtener formato amigable para LLMs desde: https://docs.crawl4ai.com/

Primero instalamos prerequisitos que requiere Google Colab.

In [22]:
!pip install h5py --quiet
!pip install typing-extensions --quiet
!pip install wheel --quiet

Posteriormente, ya podemos realizar la instalación.

In [23]:
!pip install crawl4ai --quiet
# !pip install git+https://github.com/unclecode/crawl4ai.git@2025-MAR-ALPHA-1 --quiet

In [15]:
!crawl4ai-setup

[INIT].... → Running post-installation setup...
[INIT].... → Installing Playwright browsers...
Installing dependencies...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:12 

In [16]:
!pip install nest_asyncio --quiet

In [17]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig

browser_conf = BrowserConfig(verbose=False,headless=True)

run_cfg = CrawlerRunConfig(
    wait_until="domcontentloaded",
    scan_full_page=True,
    # wait_for_timeout=2000,
    verbose=False
)

async def async_scrape_crawl4ai(url: str):
    crawler = AsyncWebCrawler(config=browser_conf)
    await crawler.start()
    result = await crawler.arun(
        url=url,
        config=run_cfg,
    )
    return result.html

# To run the async function in a synchronous context
# (like this script), you can use asyncio.run() to execute it.
# This is a workaround for running async functions in a sync context.
def scrape_crawl4ai(url: str):
    return asyncio.run(async_scrape_crawl4ai(url))

# print(scrape_crawl4ai("https://www.uv.mx"))

## Main functions to run the comparasion

Let's run all the scrapers and display them in our comparison table.

In [18]:
list_of_scraper_functions = [
      {"name": "Beautiful Soup", "function": beautiful_soup_scrape_url},
      # {"name": "Jina AI", "function": scrape_jina_ai},
      {"name": "Playwright", "function": scrape_playwright_sync},
      {"name": "Crawl4ai", "function": scrape_crawl4ai},
      ]

all_content = view_scraped_content(list_of_scraper_functions, competitor_sites, 700, 40)

Processing site UV using Crawl4ai: 100%|██████████| 1/1 [00:08<00:00,  8.55s/it]
Processing site W3C ACT Rules using Beautiful Soup: 100%|██████████| 1/1 [00:00<00:00,  6.23it/s]
Processing site W3C ACT Rules using Crawl4ai: 100%|██████████| 1/1 [00:05<00:00,  5.26s/it]


Content Table:
+---------------+------------------------------------------+------------------------------------------+------------------------------------------+
|   Site Name   |          Beautiful Soup content          |            Playwright content            |             Crawl4ai content             |
+---------------+------------------------------------------+------------------------------------------+------------------------------------------+
|     Amazon    |       5,022 characters retrieved:        |       7,040 characters retrieved:        |       7,064 characters retrieved:        |
|               |                                          |                                          |                                          |
|               |             <!DOCTYPE html>              |   <!DOCTYPE html><html class="a-no-js"   |   <!DOCTYPE html><html class="a-no-js"   |
|               |                                          |   lang="en-us"><!--<![endif]--><head>    |